In [57]:
import numpy as np
import pandas as pd
import os
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

data = pd.read_csv('chicago_crime.csv')
data.dropna(inplace=True)
data = data[['Latitude', 'Longitude', 'Date', 'Primary Type', 'Location Description', 'Description', 'Community Area']]

In [58]:
encoder = LabelEncoder()
data['Primary Type'] = encoder.fit_transform(data['Primary Type']).astype(float)
data['Description'] = encoder.fit_transform(data['Description']).astype(float)
data['Location Description'] = encoder.fit_transform(data['Location Description']).astype(float)

data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Time'] = data['Date'].dt.time

def time_to_seconds(t):
    return (datetime.timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)).total_seconds()

# Apply the function to the 'Time' column
data['Time'] = data['Time'].apply(time_to_seconds)

data = data.drop('Date', axis=1)

scaler = MinMaxScaler()
data[['Latitude', 'Longitude','Community Area','Month', 'Day', 'Time']] = scaler.fit_transform(data[['Latitude', 'Longitude','Community Area','Month', 'Day', 'Time']])

In [76]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

X_train = train_data
y_train = train_data['Primary Type']
X_val = val_data.drop(['Primary Type'], axis=1)
y_val = val_data['Primary Type']
X_test = test_data.drop(['Primary Type'], axis=1)
y_test = test_data['Primary Type']

In [82]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

#input features present in my dataset
num_features = 8
#types of crime
num_classes = 1 

model = tf.keras.models.Sequential([
    Dense(256, activation='relu', input_shape=(num_features,)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.add(Dense(16, activation='relu', input_dim=7))
model.add(Dense(8, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print(X_train)

       Latitude  Longitude  Primary Type  Location Description  Description  \
54604  0.603565   0.755937           2.0                  40.0        203.0   
37502  0.656110   0.765570          16.0                  82.0         44.0   
28385  0.638180   0.501390          28.0                 101.0          0.0   
10175  0.717674   0.510726           2.0                  98.0        203.0   
31340  0.819474   0.534605          21.0                 101.0        243.0   
...         ...        ...           ...                   ...          ...   
37591  0.224241   0.620142           5.0                  16.0        221.0   
6318   0.622275   0.519664           2.0                  87.0        203.0   
55427  0.569543   0.749149           2.0                  94.0        203.0   
865    0.291562   0.876161          25.0                  45.0        208.0   
16041  0.395114   0.760484           5.0                 101.0        225.0   

       Community Area     Month       Day      Time

In [83]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[es])

Epoch 1/100


/Users/jarraomar/.pyenv/versions/3.8.13/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1758/1777 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0865

ValueError: in user code:

    File "/Users/jarraomar/.pyenv/versions/3.8.13/lib/python3.8/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/Users/jarraomar/.pyenv/versions/3.8.13/lib/python3.8/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jarraomar/.pyenv/versions/3.8.13/lib/python3.8/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/Users/jarraomar/.pyenv/versions/3.8.13/lib/python3.8/site-packages/keras/engine/training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "/Users/jarraomar/.pyenv/versions/3.8.13/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/jarraomar/.pyenv/versions/3.8.13/